### Import libraries

In [ ]:
import os
import sys
import copy
from pathlib import Path

import numpy as np
import pandas as pd

import networkx as nx

import torch

import seaborn as sns
import matplotlib.pyplot as plt

DIR_PATH = Path.cwd().parent
sys.path.append(str(DIR_PATH))


from Models import MLP, VanillaGNN, GCN, GCONV, GAT, GIN

from Dataset.data_loader import get_data
from Dataset.preprocessing import scale_adj, pad_ohe_features, scale_feature, get_train_val_test_masks
from Dataset import DatasetTabular, DatasetGraph
from Dataset.graph import to_data_list

from Visualization.eda import histogram_classes, feature_bar_plots, violin_plots, corr_mtrx
from Visualization.graph import vis_graph, plot_graphs

from utils import set_seed, set_device

%matplotlib inline
# reload modules
%load_ext autoreload
%autoreload 2

### Fix Seeds

In [ ]:
seed=42
set_seed(seed)
device = set_device()

data_type = 'balanced' # 'unbalanced' 
DATA_PATH = DIR_PATH / 'Data'
fig_path = DIR_PATH / f'Figures/{data_type}'
os.makedirs(fig_path, exist_ok=True)

### Read the data

In [ ]:
# load the adjacency matrices and the data
A, data = get_data(DATA_PATH, data_type)  

# Print information about the dataset
S, N = A.shape[0], A.shape[1]

display(data.head(5))

# store features in x (S, N, M) and labels in y (S, 1) 
x, y = np.expand_dims(data.iloc[:, 3:].values, axis=-1), np.expand_dims(data.iloc[:,2].values, axis=-1)
M = x.shape[-1]

print(f'Number of graphs: {S}')
print(f'Number of nodes: {N}')
print(f'Number of features per node: {M}')

# Check if every graph is connected
print(f'All connected: {all(nx.is_connected(nx.from_numpy_array(G)) for G in A)}')

### EDA

In [ ]:
def get_avg_degree_centrality(A: np.array) -> np.float64:
  """
    Calculate the average degree centrality of an adjacency matrix.
  """
  return np.mean(list(dict(nx.degree_centrality(nx.from_numpy_array(A))).values()))

def get_avg_closeness_centrality(A: np.array) -> np.float64:
  """
    Calculate the average closeness centrality of an adjacency matrix.
  """
  return np.mean(list(dict(nx.closeness_centrality(nx.from_numpy_array(A))).values()))

def get_avg_betweenness_centrality(A: np.array) -> np.float64:
  """
    Calculate the average betweenness centrality of an adjacency matrix.
  """
  return np.mean(list(dict(nx.betweenness_centrality(nx.from_numpy_array(A))).values()))

In [ ]:
# Store the adjacency matrices for each of the classes
A_pos = A[np.where(y==0)[0]]
A_neg = A[np.where(y==1)[0]]

# Compute average degree centrality for each sample per class
deg_pos_mean = [get_avg_degree_centrality(a) for a in A_pos]
deg_neg_mean = [get_avg_degree_centrality(a) for a in A_neg]

# Compute average closeness centrality for each sample per class
cls_pos_mean = [get_avg_closeness_centrality(a) for a in A_pos]
cls_neg_mean = [get_avg_closeness_centrality(a) for a in A_neg]

# Compute average betweenness centrality for each sample per class
btw_pos_mean = [get_avg_betweenness_centrality(a) for a in A_pos]
btw_neg_mean = [get_avg_betweenness_centrality(a) for a in A_neg]


# Combine data and metrics into lists
pos_data = [deg_pos_mean, cls_pos_mean, btw_pos_mean]
neg_data = [deg_neg_mean, cls_neg_mean, btw_neg_mean]
metrics = ['Degree Centrality', 'Closeness Centrality', 'Betweenness Centrality']
binranges = [(0.2, 0.5), (0.45, 0.75), (0, 0.1)]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
# Iterate over the data and metrics to create each subplot
for i, (pos, neg, metric, binrange) in enumerate(zip(pos_data, neg_data, metrics, binranges)):
        histogram_classes(pos, neg, metric, ax=axes[i], bins=50, binrange=binrange)  

fig.tight_layout()  # Adjust subplots to fit into figure area.
fig.savefig(fig_path / 'histogram.png')  # Save the figure before showing it.

Apparently, we can already differentiate between positive and negative based on just the graph structure. **A perfect classifier just using a threshold**, do we even need any GNNs after this? We will further analyze the node features to see if there is a similar way to distinguish between the classes and apply some SOTA GNN models.

In [ ]:
# plot the bars of mean and with std values
x_pos = x[np.where(y==0)[0]].squeeze()
x_neg = x[np.where(y==1)[0]].squeeze()

save_path = fig_path / 'bar_plot.png'
feature_bar_plots(x_pos, x_neg, save_path)

In [ ]:
# Plot the violin plot of the features
df_melted = pd.melt(data, id_vars=['class'], 
                    value_vars=[f'Prot{i}' for i in range(1, N+1)], 
                    var_name='Protein', value_name='Expression')
                                                                                                                                                                                                                                                                                                                                                                                                   
save_path = fig_path / 'violins.png'
violin_plots(df_melted, save_path)

In [ ]:
# Plot the correlation between the features
corr = data.iloc[:, 2:].corr()

save_path = fig_path / 'correlation.png'
corr_mtrx = corr_mtrx(corr, save_path)

The features seem to have a Gaussian distribution. They do not provide a strong correlation with the **class** target, although some features, such as **Prot18**, **Prot11** seem to be more correlated, which is confirmed by the violin plats, which show a small difference in the mean values per class.

In [ ]:
# Visualize one graph and color the nodes according to the feature values
idx = 0
label = 'Positive' if y[idx]==1 else 'Negative'

print(f'Graph {idx+1}, {label} class')

save_path = fig_path / f'graph_{label}.png'
vis_graph(A[idx], x[idx, :, 0], label, save_path)


In [ ]:
# Plot a few graphs of both classes together
plot_graphs(A, x, y, 9, 'Positive')
plot_graphs(A, x, y, 9, 'Negative')

## Data Preprocessing

In [ ]:
SCALE_ADJ_WEIGHTS = True
ADD_OHE_FEATURES = True
NORMALIZE_FEATURES = True 

if SCALE_ADJ_WEIGHTS:
    # Scale the weights of the adjacency matrices
    A = scale_adj(A)    

if ADD_OHE_FEATURES:
    # Add the identifiers of each node as a OHE feature
    x = pad_ohe_features(x, N)
    # Number of features is increased by the number of nodes
    M += N

# Split the data into train, validation, and test sets
train_mask, val_mask, test_mask = get_train_val_test_masks(S, y)

if NORMALIZE_FEATURES: # TODO: Check this 
    # Standardize the features from a list features_id 
    features_id = [0] # list of features (columns) you want to scale
    for feature_id in features_id:
        x = scale_feature(x, feature_id, train_mask, val_mask, test_mask)
        
         # check the distribution
        data_scaled = np.hstack((x[:, :, feature_id], y))
        
        df = pd.DataFrame(data=data_scaled, columns=list(data.columns[3:]) + [data.columns[2]]) # set the columns in the cirrect order
        df_melted1 = pd.melt(df, id_vars=['class'], 
                            value_vars=[f'Prot{i}' for i in range(1, 21)], 
                            var_name='Protein', value_name='Expression')
        
        save_path = fig_path / f'violins_norm_{feature_id}.png'                                                                                                                                                                                                                                                                                                                                                              
        violin_plots(df_melted1, save_path)


In [ ]:
# TODO: ADD SMTH ABOOUT THE FEATRURES

## Deep Learning Graph Classification

### MLP
As a baseline model, we can firs us a classic MLP architecture, which completely ignores any topology of the dataset.

In [ ]:

# Create datasets and convert to mini-batches
train_loader = DatasetTabular(x[train_mask], y[train_mask], device).to_loader(batch_size=64, shuffle=True)
val_loader = DatasetTabular(x[val_mask], y[val_mask], device).to_loader(batch_size=64, shuffle=True)
test_loader = DatasetTabular(x[test_mask], y[test_mask], device).to_loader(batch_size=64, shuffle=True)

In [ ]:
#N_sim = 5
#epochs = 120

def test_n_times(model_, train_loader, val_loader, test_loader, epochs, N_sim, verbose=True):
    """Test the model N_times and return the average accuracy."""
    accs = []
    losses = []
    for i in range(N_sim):
    # Create a copy of the model
        model = copy.deepcopy(model_) 
        # Train
        model.fit(train_loader, val_loader, epochs=epochs, verbose=verbose)
        # Test
        loss, acc = model.test(test_loader)
        if verbose:
            print(f'Trial {i+1}, test loss: {loss:.2f} | test accuracy: {acc*100:.2f}%')
        
        # Store the results
        accs.append(acc.detach().detach().cpu().numpy())
        losses.append(loss.detach().detach().cpu().numpy())

    losses, accs = np.array(losses), np.array(accs)
    
    print(f'\nFinal accuracy: {np.mean(accs)*100:.2f}%, std: {np.std(accs)*100:.2f}%')
    print(f'Final loss: {np.mean(losses):.2f}, std: {np.std(losses):.2f}%')

In [ ]:
mlp = MLP(dim_in=N*M, dim_h=16, num_layers=1).to(device)
print(mlp)
test_n_times(mlp, train_loader, val_loader, test_loader, 300, 1, verbose=True)

### Prepare a graph dataset

In [ ]:
data_list = to_data_list(A, x, y, device)
dataset = DatasetGraph(root='../Data', data_list=data_list) # delete the folder to reset

train_loader = dataset[train_mask].to_loader(batch_size=64, shuffle=True)
val_loader   = dataset[val_mask].to_loader(batch_size=64, shuffle=True)
test_loader  = dataset[test_mask].to_loader(batch_size=64, shuffle=True)

### Vanilla GNN

In [ ]:
gnn = VanillaGNN(dim_in=M, dim_h=16, num_layers=1).to(device)
print(gnn)
test_n_times(gnn, train_loader, val_loader, test_loader, 500, 1, verbose=True)

### GCN

In [ ]:
gcn = GCN(dim_in=M, dim_h=16, num_layers=2, dropout=True).to(device) # weights
print(gcn)
test_n_times(gcn, train_loader, val_loader, test_loader, 300, 1, verbose=True)

### GraphConv

In [ ]:
gcn = GCONV(dim_in=M, dim_h=16, num_layers=1).to(device)
print(gcn)
test_n_times(gcn, train_loader, val_loader, test_loader, 500, 1, verbose=True)

### GAT

In [ ]:
gat = GAT(dim_in=M, dim_h=16, num_layers=2).to(device)
print(gat)
test_n_times(gat, train_loader, val_loader, test_loader, 500, 1, verbose=True)

### GIN

In [ ]:
gin = GIN(dim_in=M, dim_h=16, num_layers=1).to(device)
print(gin)
test_n_times(gin, train_loader, val_loader, test_loader, 500, 1, verbose=True)

In [ ]:
dim = 32

from tqdm import tqdm
from node2vec import Node2Vec

x_ = torch.concat((torch.tensor(x, dtype=torch.float32), torch.zeros((x.shape[0], x.shape[1], dim), dtype=torch.float32)), dim=-1).to(device)

for i in tqdm(range(A.shape[0])):
    # embedding for each graph
    G = nx.from_numpy_array(A[i])
    node2vec = Node2Vec(G, dimensions=dim, walk_length=20, num_walks=100, p=2, q=1, workers=20, quiet=True)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    x_[i, :, 1:] = torch.tensor(model.wv[np.arange(N)], dtype=torch.float32)
    
    #print(model.wv[np.arange(N)].shape)

Similar outpcomes are found in this **stack overflow** discussion [here](https://stackoverflow.com/questions/75752422/gnn-graph-classification-poor-performance)

### Performance comparison

### GNNExplainer and IntegratedGradients

In [ ]:
#from torch_geometric.data import Data
#from torch_geometric.loader import DataLoader
#from scipy.sparse import coo_matrix

#def get_data(dataset, idx):
#    return list(map(dataset.__getitem__, idx))

#dataset = []
#for idx in range(len(A)):
#    edge_index = coo_matrix(A[idx])
#    sample = Data(x=x[idx], edge_index=np.vstack((edge_index.row, edge_index.col)), edge_weight=edge_index.data, y=y[idx])
#    dataset.append(sample)

#train_dataset = get_data(dataset, id_train) 
#val_dataset = get_data(dataset, id_val) 
#test_dataset = get_data(dataset, id_test) 

#print(f"Number of samples: Training {len(train_dataset)} | Validation {len(val_dataset)} | Testing {len(test_dataset)}")   

#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:

# Train standard scaler on training and validation data, then apply to test data
#SCALE_DATA = True # Scale or use raw features

#if SCALE_DATA:
#    scaler = StandardScaler()
#    # train the scaler on training and validation
#    scaler.fit(np.concatenate((x_train, x_val), axis=0).reshape(-1, M)) 
#    # apply the scaler to the data (prevent information leak to the test data)
#    print(x_train[0][0])
#    x_train = scaler.transform(x_train.reshape(-1, M)).reshape(x_train.shape)
#    print(x_train[0][0])
#    x_val = scaler.transform(x_val.reshape(-1, M)).reshape(x_val.shape)
#    X_test = scaler.transform(x_test.reshape(-1, M)).reshape(x_test.shape)

    # check the distribution
#    data_scaled = np.hstack((
#                            np.concatenate((x_train, x_val, x_test), axis=0).squeeze(), 
#                            np.concatenate((y_train, y_val, y_test))
#                            )
#                        )
#    df = pd.DataFrame(data=data_scaled, columns=list(data.columns[3:]) + [data.columns[2]]) # set the columns in the cirrect order
#    df_melted1 = pd.melt(df, id_vars=['class'], 
#                        value_vars=[f'Prot{i}' for i in range(1, 21)], 
#                        var_name='Protein', value_name='Expression')
#                                                                                                                                                                                                                                                                                                                                                                                                    
#    sns.set(style="whitegrid")
#    palette = {1: 'skyblue', 0: 'salmon'}
#    plt.figure(figsize=(10, 8))
#    ax = sns.violinplot(x='Protein', y='Expression', hue='class', data=df_melted1, split=True, inner="quart", palette=palette)
#    ax.legend(handles=ax.legend_.legendHandles, labels=['Negative', 'Positive'])
#    plt.axhline(y=0, color='b', linestyle='--')
#    plt.title('Violin Plots per Class (Scaled)')
#    plt.xlabel('Feature (Node)')
#    plt.ylabel('Value')
#    plt.xticks(rotation=45)
#    plt.ylim([-11, 11])
#    plt.show()

In [ ]:
#N_sim = 20 # number of you simulate to get the average performance
#verbose = True
#res_mlp = []
#for i in range(N_sim):
#    # Create MLP model
#    mlp = MLP(dim_in=N*M, dim_h=16).to(device)
#    if i == 0:
#        print(mlp,'\n')
    
#    # Train
#    mlp.fit(data_mlp, epochs=150, verbose=verbose)
    
#    # Test
#    acc = mlp.test(data_mlp)
#    if verbose:
#        print(f'MLP test accuracy: {acc*100:.2f}%')
    
#    # Store the results
#    res_mlp.append(acc.detach().detach().cpu().numpy())

#res_mlp = np.array(res_mlp)
#print(f'\nMLP final accuracy: {np.mean(res_mlp)*100:.2f}%, std: {np.std(res_mlp)*100:.2f}%')